In [ ]:
%%capture
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.stats as stats

from dj_notebook import activate

plus = activate(dotenv_file="/Users/erikvw/source/edc_source/meta-edc/.env")
# output is suppressed ut normally would spew out all the edc loading messages


In [ ]:
# This notebook is incomplete / not working

In [ ]:
from edc_analytics.custom_tables import BpTable
from edc_analytics.table import Table
from meta_screening.models import SubjectScreening
from meta_subject.models import PhysicalExam, SubjectVisit
from django_pandas.io import read_frame

In [ ]:
default_columns = ["id", "subject_identifier", "report_datetime", "visit_code"]

title_row = []  # ???????????????

In [ ]:
# this step is slow, maybe because it is the first call to the DB
qs_screening = SubjectScreening.objects.all()
df = read_frame(qs_screening)


In [ ]:
# backup the df
df_screen = df.copy()
# df = df_screen.copy()


In [ ]:
# convert all to float
cols = ["fbg_value", "fbg2_value", "ogtt_value", "ogtt2_value", "converted_fbg_value", 
        "converted_fbg2_value", "converted_ogtt_value", "converted_ogtt2_value",
       "sys_blood_pressure_avg", "dia_blood_pressure_avg",
       "waist_circumference"]
df[cols] = df[cols].apply(pd.to_numeric)

In [ ]:

# condition to include any glucose test
cond_glu = (
    (df['fbg_value'].notna()) | 
    (df['ogtt_value'].notna()) | 
    (df['fbg2_value'].notna()) |
    (df['ogtt2_value'].notna())
)

# conditions for Male/Female
male = (df["gender"]=="Male")
female = (df["gender"]=="Female")

# condition for art stable
cond_art_stable = (df['on_rx_stable']=="Yes") & (df['vl_undetectable']=="Yes") & (df['art_six_months']=="Yes") 


In [ ]:
# lets fix some columns
# has_dm fillna with unk
df["has_dm"] = df["has_dm"].apply(lambda x: "unk" if not x else x)
# lets create a column that summarizes lives_nearby and staying_nearby_12
df["in_catchment"] = (df["lives_nearby"] == "Yes") & (df["staying_nearby_12"] == "Yes")


# glucose
# are all glucose fields filled? YES
# for prefix in ["fbg", "ogtt", "fbg2", "ogtt2"]:
#     print(df[(df[f"{prefix}_value"].isna()) & (df[f"converted_{prefix}_value"].notna())]["gender"].count())
#     print(df[(df[f"{prefix}_value"].notna()) & (df[f"converted_{prefix}_value"].isna())]["gender"].count())

# create fbg column
df["fbg"] = df["converted_fbg_value"]
df.loc[df["fbg"].notna() & df["converted_fbg2_value"].notna(), "fbg"] = df["converted_fbg2_value"]

# create ogtt column
df["ogtt"] = df["converted_ogtt_value"]
df.loc[df["ogtt"].notna() & df["converted_ogtt2_value"].notna(), "ogtt"] = df["converted_ogtt2_value"]


In [ ]:
# subject SR9E8B4D has eligible part two == No but subject has a glucose value
df.loc[(df["screening_identifier"]=="SR9E8B4D"), "eligible_part_two"] = "Yes"


In [ ]:

# condition where subject is eligible P1/P2 and has any type of glucose test
cond = ((df["eligible_part_one"]=="Yes") & (df["eligible_part_two"]=="Yes") & cond_glu)


In [ ]:
# filter dataframe
df = df[cond]


In [ ]:
print(len(df))

In [ ]:
wc_describe = df["waist_circumference"].describe()

# merge with physical exam to get waist circumference if taken at baseline
subject_identifiers = list(df["subject_identifier"])

qs_subject_visit = SubjectVisit.objects.filter(subject_identifier__in=subject_identifiers)
df_subject_visit = read_frame(qs_subject_visit)
df_subject_visit.rename(columns={"id": "subject_visit"}, inplace=True)

qs_physical_exam = PhysicalExam.objects.filter(subject_visit__subject_identifier__in=subject_identifiers)
df_physical_exam = read_frame(qs_physical_exam)

# merge w/ subject visit to get subject_identifier
df_physical_exam = pd.merge(df_physical_exam, df_subject_visit[["subject_visit", "subject_identifier", "visit_code", "visit_code_sequence"]], on="subject_visit", how="left")
df_physical_exam = df_physical_exam[["subject_identifier", "visit_code", "visit_code_sequence", "waist_circumference"]]

df_physical_exam[["waist_circumference"]] = df[["waist_circumference"]].apply(pd.to_numeric)

# rename column to waist_circumference_baseline
df_physical_exam["waist_circumference_baseline"] = df_physical_exam["waist_circumference"]
df_physical_exam.drop(columns=["waist_circumference"])

df_physical_exam[["waist_circumference_baseline"]] = df_physical_exam[["waist_circumference_baseline"]].apply(pd.to_numeric)
wc_baseline_describe = df_physical_exam["waist_circumference_baseline"].describe()

# merge on subject_identifier with main DF
df = pd.merge(df, df_physical_exam[["subject_identifier", "waist_circumference_baseline"]], on="subject_identifier", how="left")

# set waist_circumference=waist_circumference_baseline if `waist_circumference` is none and `waist_circumference_baseline` is not
df.loc[(df["waist_circumference"].isna()) & (df["waist_circumference_baseline"].notna()), "waist_circumference"] = df["waist_circumference_baseline"]

# drop waist_circumference_baseline
df.drop(columns=["waist_circumference_baseline"], inplace=True)


In [ ]:
# gender
def cell(gender, all=None):
    cnt = df.loc[gender]["gender"].count()
    if not all:
        tot = df["gender"].count()
        return f"{cnt} ({round(cnt/tot *100, 1)}%)"
    return f"{cnt}"

df_gender = pd.DataFrame(columns=default_columns)
class GenderTable(Table):
    def build_table_df(self):
        pass

tbl = Table(df, label="Gender", columns=default_columns, show_ncol_perc=True)
# df_gender.loc[0] = ["Gender", "n", cell(female), cell(male), cell((male | female), all=True)]
tbl.table_df


In [ ]:
# age
agef = df.loc[female]["age_in_years"]
agem = df.loc[male]["age_in_years"]
age = df["age_in_years"]
# bins
bin1 = (df["age_in_years"]>=18) & (df["age_in_years"]<35)
bin2 = (df["age_in_years"]>=35) & (df["age_in_years"]<50)
bin3 = (df["age_in_years"]>=50) & (df["age_in_years"]<65)
bin4 = (df["age_in_years"]>=65)

def cell(cond, gender, all=None):
    cnt = df.loc[gender & cond]["age_in_years"].count()
    if not all:
        tot = df.loc[cond]["age_in_years"].count()
        return f"{cnt} ({round(cnt/tot *100, 1)}%)"
    tot = df["age_in_years"].count()
    return f"{cnt} ({round(cnt/tot *100, 1)}%)"

df_age = pd.DataFrame(columns=title_row)

df_age.loc[0] = ["Age (years)", "n", agef.count(), agem.count(), age.count()]
df_age.loc[1] = [
    "", "Median (IQR)",
    f"{agef.quantile().astype(int)} ({agef.quantile(0.25).astype(int)}, {agef.quantile(0.75).astype(int)})",
    f"{agem.quantile().astype(int)} ({agem.quantile(0.25).astype(int)}, {agem.quantile(0.75).astype(int)})",
    f"{age.quantile().astype(int)} ({age.quantile(0.25).astype(int)}, {age.quantile(0.75).astype(int)})",]    
df_age.loc[2] = ["", "18-34", cell(female, bin1), cell(male, bin1), cell(bin1, (male | female), all=True)]
df_age.loc[3] = ["", "35-49", cell(female, bin2), cell(male, bin2), cell(bin2, (male | female), all=True)]
df_age.loc[4] = ["", "50-64", cell(female, bin3), cell(male, bin3), cell(bin3, (male | female), all=True)]
df_age.loc[5] = ["", "65 and older", cell(female, bin4), cell(male, bin4), cell(bin4, (male | female), all=True)]


In [ ]:
# waist_circumference
desc = df[["waist_circumference"]].describe()
descf = df[df["gender"]=="Female"][["waist_circumference"]].describe()
descm = df[df["gender"]=="Male"][["waist_circumference"]].describe()

f = f"{descf.loc["50%"].values[0]} ({descf.loc["25%"].values[0]}, {descf.loc["75%"].values[0]})"
m = f"{descm.loc["50%"].values[0]} ({descm.loc["25%"].values[0]}, {descm.loc["75%"].values[0]})"
all = f"{desc.loc["50%"].values[0]} ({desc.loc["25%"].values[0]}, {desc.loc["75%"].values[0]})"

df_waist = pd.DataFrame(columns=title_row)

df_waist.loc[0] = ["Waist circumference (cm)", "n", descf.loc["count"].values[0].astype("int64"), descm.loc["count"].values[0].astype("int64"), desc.loc["count"].values[0].astype("int64")]
df_waist.loc[1] = ["", "Median (IQR)", f, m, all]


In [ ]:
# waist_circumference (cont)
# Women 88 / Men 102
cond_lt_102 = ((df["waist_circumference"]<102.0) & (df["gender"]=="Male")) | ((df["waist_circumference"]<88.0) & (df["gender"]=="Female"))
cond_gte_102 = ((df["waist_circumference"]>=102.0) & (df["gender"]=="Male")) | ((df["waist_circumference"]>=88.0) & (df["gender"]=="Female"))

tot = df["waist_circumference"].count()

f_cnt = df[cond_lt_102 & female]["waist_circumference"].count()
f = f"{round(f_cnt, 0)} ({round(f_cnt/tot * 100, 1)}%)"
m_cnt = df[cond_lt_102 & male]["waist_circumference"].count()
m = f"{round(m_cnt, 0)} ({round(m_cnt / tot * 100, 1) }%)"
value = f"{round(df[cond_lt_102]["waist_circumference"].count(), 3)} ({round(df[cond_lt_102]["waist_circumference"].count() / df["waist_circumference"].count(), 3) * 100}%)"


df_waist.loc[2] = ["", "Women<88 / Men<102", f, m, value]


f_cnt = df[cond_gte_102 & female]["waist_circumference"].count()
f = f"{round(f_cnt, 0)} ({round(f_cnt/tot * 100, 1)}%)"
m_cnt = df[cond_gte_102 & male]["waist_circumference"].count()
m = f"{round(m_cnt, 0)} ({round(m_cnt / tot * 100, 1) }%)"
value = f"{round(df[cond_gte_102]["waist_circumference"].count(), 3)} ({round(df[cond_gte_102]["waist_circumference"].count() / df["waist_circumference"].count(), 3) * 100}%)"

df_waist.loc[3] = ["", "Women>=88 / Men>=102", f, m, value]
                   

In [ ]:
# cond_art

def cell(gender, all=None):
    cnt = df.loc[gender & cond_art_stable]["gender"].count()
    if not all:
        tot = df.loc[cond_art_stable]["gender"].count()
        return f"{cnt} ({round(cnt/tot *100, 1)}%)"
    tot = df["gender"].count()
    return f"{cnt} ({round(cnt/tot *100, 1)}%)"
    
df_art = pd.DataFrame(columns=title_row)
df_art.loc[0] = ["Stable on ART", "", cell(female), cell(male), cell((male | female), all=True)]


In [ ]:
# blood pressure
# print(len(df[(df["sys_blood_pressure_one"].notna()) & (df["dia_blood_pressure_one"].notna())]))
# print(len(df[(df["sys_blood_pressure_two"].notna()) & (df["dia_blood_pressure_two"].notna())]))
# print(len(df[(df["sys_blood_pressure_avg"].notna()) & (df["dia_blood_pressure_avg"].notna())]))

In [ ]:
# blood pressure

# df_tmp = df.copy()
# tot = len(df_tmp)
# print(f"tot={tot}")
# len(df_tmp[(df_tmp["sys_blood_pressure_avg"].notna()) & (df_tmp["dia_blood_pressure_avg"].notna())])
# severe_htn_cond = (df_tmp["sys_blood_pressure_avg"]>=180) | (df_tmp["dia_blood_pressure_avg"]>=110)
# severe_htn_df = df_tmp[severe_htn_cond]
# print(f"severe_htn={len(severe_htn_df)}")
# df_tmp.drop(severe_htn_df.index, inplace=True)

# htn_cond = (df_tmp["sys_blood_pressure_avg"]>=140) | (df_tmp["dia_blood_pressure_avg"]>=90)
# htn_df = df_tmp[htn_cond]
# print(f"htn={len(htn_df)}")
# df_tmp.drop(htn_df.index, inplace=True)

# pre_htn_cond = (df_tmp["sys_blood_pressure_avg"]>=120) | (df_tmp["dia_blood_pressure_avg"]>=80)
# pre_htn_df = df_tmp[pre_htn_cond]
# print(f"pre_htn={len(pre_htn_df)}")
# df_tmp.drop(pre_htn_df.index, inplace=True)

# normal_cond = (df_tmp["sys_blood_pressure_avg"]>=90) | (df_tmp["dia_blood_pressure_avg"]>=60)
# normal_df = df_tmp[normal_cond]
# print(f"normal={len(normal_df)}")
# df_tmp.drop(normal_df.index, inplace=True)

# low_cond = (df_tmp["sys_blood_pressure_avg"]>=0) | (df_tmp["dia_blood_pressure_avg"]>=0)
# low_df = df_tmp[low_cond]
# print(f"low={len(low_df)}")
# df_tmp.drop(low_df.index, inplace=True)

    
# def cell(dfx, gender, all=None, perc=True):
#     cnt = dfx.loc[gender]["gender"].count()
#     if not all:
#         tot = df.loc[gender]["gender"].count()
#         return f"{cnt}" if not perc else f"{cnt} ({round(cnt/tot *100, 1)}%)" 
#     tot = df["gender"].count()
#     return f"{cnt}" if not perc else f"{cnt} ({round(cnt/tot *100, 1)}%)"

# def quantile(gender, colname):
#     q50 = df.loc[gender][colname].quantile()
#     q25 = df.loc[gender][colname].quantile(0.25)
#     q75 = df.loc[gender][colname].quantile(0.75)
#     return f"{q50} ({q25}, {q75})"

# df_bp = pd.DataFrame(columns=title_row)
# df_bp.loc[0] = ["Blood pressure at baseline (mmHg)", "n", cell(df_tmp, female), cell(df_tmp, male), cell(df_tmp, (male | female), all=True, perc=False)]
# df_bp.loc[1] = ["", "Low (<90/60)", cell(low_df, female), cell(low_df, male), cell(low_df, (male | female), all=True)]
# df_bp.loc[2] = ["", "Normal (<120/80)", cell(normal_df, female), cell(normal_df, male), cell(normal_df, (male | female), all=True)]
# df_bp.loc[3] = ["", "Pre-hypertension (<140/90)", cell(pre_htn_df, female), cell(pre_htn_df, male), cell(pre_htn_df, (male | female), all=True)]
# df_bp.loc[4] = ["", "Hypertension (>=140/90)", cell(htn_df, female), cell(htn_df, male), cell(htn_df, (male | female), all=True)]
# df_bp.loc[5] = ["", "Severe hypertension (>=180/110)", cell(severe_htn_df, female), cell(severe_htn_df, male), cell(severe_htn_df, (male | female), all=True)]
# df_bp.loc[6] = ["", "Systolic - median (IQR)", quantile(female, "sys_blood_pressure_avg"), quantile(male, "sys_blood_pressure_avg"), quantile((female | male), "sys_blood_pressure_avg")]
# df_bp.loc[7] = ["", "Diastolic - median (IQR)", quantile(female, "dia_blood_pressure_avg"), quantile(male, "dia_blood_pressure_avg"), quantile((female | male), "dia_blood_pressure_avg")]



In [ ]:
# columns = [
#     'Characteristics', 'Statistics', "F", "M", 'All', 
#     "fnum","f_prop","fq25","fq50","fq75",
#     "mnum","m_prop","mq25","mq50","mq75",
#      "q25","q50","q75","tot"]

# class SubjectRow:
#     def __init__(self, gender, dfx, main_df, iqr_col=None):
#         self.num = dfx.loc[gender]["gender"].count()
#         self.total = len(main_df.loc[gender])
#         self.perc = self.num/self.total
#         if iqr_col:
#             self.q25, self.q50, self.q75 = dfx.loc[gender][iqr_col].quantile([0.25, 0.50, 0.75])
#         else:
#             self.q25, self.q50, self.q75 = np.nan,np.nan,np.nan

# class MaleRow(SubjectRow):
#     def __init__(self, dfx, main_df, iqr_col=None):
#         super().__init__(male, dfx, main_df, iqr_col)

# class FemaleRow(SubjectRow):
#     def __init__(self, dfx, main_df, iqr_col=None):
#         super().__init__(female, dfx, main_df, iqr_col)

# class Row:
#     def __init__(self, dfx, main_df, label=None, statistic=None, iqr_col=None, columns=None):
#         self.m = MaleRow(dfx, main_df, iqr_col)
#         self.f =FemaleRow(dfx, main_df, iqr_col)
#         self.total = len(main_df)
#         self.subtotal = len(dfx)
#         if iqr_col:
#             self.q25, self.q50, self.q75 = main_df[iqr_col].quantile([0.25, 0.50, 0.75])
#         else:
#             self.q25, self.q50, self.q75 = np.nan,np.nan,np.nan        
#         self.label = label or ""
#         self.statistic = statistic
#         self.df = pd.DataFrame(columns=columns)

#     def with_perc(total=None):
#         if total:
#             return f"{self.num} ({round(self.num/self.total *100, 1)}%)"
#         return f"{self.num} ({round(self.num/self.subtotal *100, 1)}%)"
    
#     def values(self):
#         if self.statistic=="n":
#             return [
#                 self.label, self.statistic, "", "", "",
#                 self.f.num, self.f.perc, self.f.q25, self.f.q50, self.f.q75,
#                 self.m.num, self.m.perc, self.m.q25, self.m.q50, self.m.q75,
#                 self.q25, self.q50, self.q75, 
#                 self.total]
#         return [
#             self.label, self.statistic, "", "", "", 
#             self.f.num, self.f.perc, self.f.q25, self.f.q50, self.f.q75, 
#             self.m.num, self.m.perc,self.m.q25, self.m.q50, self.m.q75, 
#             self.q25, self.q50, self.q75, 
#             self.subtotal]

# class Table:

#     statistic_col = "Statistics"
#     female_col = "F"
#     male_col = "M"
#     all_col = "All"
#     n_sublabel = "n"
#     grand_total_col = "tot"
    
#     def __init__(self, main_df, label=None, columns=None):
#         self.main_df = main_df
#         self.table_df = pd.DataFrame(columns=columns)
#         self.row_zero = Row(main_df, main_df, label=label, statistic=self.n_sublabel, columns=columns)

#         self.build_table_df()
        
#         # format string cols
#         self.table_df[self.female_col] = self.table_df.apply(lambda x: self.format_f_col(x), axis=1)
#         self.table_df[self.male_col] = self.table_df.apply(lambda x: self.format_m_col(x), axis=1)
#         self.table_df[self.all_col] = self.table_df.apply(lambda x: self.format_all_col(x), axis=1)

#     def build_table_df(self):
#         self.table_df.loc[0] = self.row_zero.values()

#     @property
#     def formatted_df(self):
#         return self.table_df[['Characteristics', 'Statistics', "F", "M", 'All']]

#     def format_f_col(self, x):
#         if x[self.statistic_col] == self.n_sublabel:
#             return f"{x.fnum}"
#         elif pd.notna(x.q25):
#             return f"{x.fq50} ({x.fq25},{x.fq75})"
#         return f"{x.fnum} ({round(x.fnum/self.row_zero.f.total *100, 1)}%)" 

#     def format_m_col(self, x):
#         if x[self.statistic_col] == self.n_sublabel:
#             return f"{x.mnum}"
#         elif pd.notna(x.q25):
#             return f"{x.mq50} ({x.mq25},{x.mq75})"
#         return f"{x.mnum} ({round(x.mnum/self.row_zero.m.total *100, 1)}%)" 

#     def format_all_col(self, x):
#         if x[self.statistic_col] == self.n_sublabel:
#             return f"{x.tot}"
#         elif pd.notna(x.q25):
#             return f"{x.q50} ({x.q25},{x.q75})"
#         return f"{x.tot} ({round(x.tot/self.table_df.loc[0][self.grand_total_col] *100, 1)}%)" 

# class BpTable(Table):

#     sys_col = "sys_blood_pressure_avg"
#     dia_col = "dia_blood_pressure_avg"

#     def build_table_df(self):
#         self.table_df.loc[0] = self.row_zero.values()
#         i = 1
#         for key, dfx in self.get_dfs(self.main_df).items():
#             self.table_df.loc[i] = Row(dfx, self.main_df, label="", statistic=key, columns=columns).values()
#             i += 1
#         self.table_df.loc[i+1] = Row(self.main_df, self.main_df, label="", statistic="Systolic - median (IQR)", iqr_col="sys_blood_pressure_avg", columns=columns).values()
#         self.table_df.loc[i+2] = Row(self.main_df, self.main_df, label="", statistic="Diastolic - median (IQR)", iqr_col="dia_blood_pressure_avg", columns=columns).values()        
    
#     def get_dfs(self, main_df):
#         dfs = {}
#         df_tmp = main_df.copy()
#         tot = len(df_tmp)
#         severe_htn_cond = (df_tmp["sys_blood_pressure_avg"]>=180) | (df_tmp["dia_blood_pressure_avg"]>=110)
#         severe_htn_df = df_tmp[severe_htn_cond]
#         dfs.update({"Severe hypertension (>=180/110)": severe_htn_df})
#         df_tmp.drop(severe_htn_df.index, inplace=True)
        
#         htn_cond = (df_tmp["sys_blood_pressure_avg"]>=140) | (df_tmp["dia_blood_pressure_avg"]>=90)
#         htn_df = df_tmp[htn_cond]
#         dfs.update({"Hypertension (>=140/90)": htn_df})
#         df_tmp.drop(htn_df.index, inplace=True)
        
#         pre_htn_cond = (df_tmp["sys_blood_pressure_avg"]>=120) | (df_tmp["dia_blood_pressure_avg"]>=80)
#         pre_htn_df = df_tmp[pre_htn_cond]
#         dfs.update({"Pre-hypertension (<140/90)": pre_htn_df})
#         df_tmp.drop(pre_htn_df.index, inplace=True)
        
#         normal_cond = (df_tmp["sys_blood_pressure_avg"]>=90) | (df_tmp["dia_blood_pressure_avg"]>=60)
#         normal_df = df_tmp[normal_cond]
#         dfs.update({"Normal (<120/80)": normal_df})
#         df_tmp.drop(normal_df.index, inplace=True)
        
#         low_cond = (df_tmp["sys_blood_pressure_avg"]>=0) | (df_tmp["dia_blood_pressure_avg"]>=0)
#         low_df = df_tmp[low_cond]
#         dfs.update({"Low (<90/60)": low_df})
#         df_tmp.drop(low_df.index, inplace=True)
#         dfs = dict(reversed(list(dfs.items())))
#         return dfs

    

tbl = BpTable(df, label="Blood pressure at baseline (mmHg)", columns=columns)
tbl.formatted_df
        

In [ ]:
tbl.table_df

In [ ]:
df_bp2

In [ ]:
# fbg

def cell(measure, gender, all=None):
    if measure == "<6.1":
        cond  = (df["fbg"]<6.1)
    elif measure == "6.1-6.9":
        cond  = (df["fbg"]>=6.1) & (df["fbg"]<7.0)
    elif measure == ">=7.0":
        cond  = (df["fbg"]>=7.0)
    else:
        cond = (df["fbg"].notna())
    cnt = df.loc[gender & cond]["gender"].count()
    if not all:
        tot = df.loc[gender]["gender"].count()
        return f"{cnt}" if measure=="n" else f"{cnt} ({round(cnt/tot *100, 1)}%)" 
    tot = df["gender"].count()
    return f"{cnt}" if measure=="n" else f"{cnt} ({round(cnt/tot *100, 1)}%)"

df_fbg = pd.DataFrame(columns=title_row)
df_fbg.loc[0] = ["FBG (mmol/L) categories", "n", cell("n", female), cell("n", male), cell("n", (male | female), all=True)]
df_fbg.loc[1] = ["", "<6.1", cell("<6.1", female), cell("<6.1", male), cell("<6.1", (male | female), all=True)]
df_fbg.loc[2] = ["", "6.1-6.9", cell("6.1-6.9", female), cell("6.1-6.9", male), cell("6.1-6.9", (male | female), all=True)]
df_fbg.loc[3] = ["", "7.0 and above", cell(">=7.0", female), cell(">=7.0", male), cell(">=7.0", (male | female), all=True)]



In [ ]:
# ogtt

def cell(measure, gender, all=None):
    if measure == "<7.7":
        cond  = (df["ogtt"]<7.8)
    elif measure == "7.8-11.1":
        cond  = (df["ogtt"]>=7.8) & (df["ogtt"]<11.1)
    elif measure == ">=11.1":
        cond  = (df["ogtt"]>=11.1)
    elif measure == "missing":
        cond = (df["ogtt"].isna())
    else:
        cond = (df["ogtt"].notna() | df["ogtt"].isna())
    cnt = df.loc[gender & cond]["gender"].count()
    if not all:
        tot = df.loc[gender]["gender"].count()
        return f"{cnt}" if measure=="n" else f"{cnt} ({round(cnt/tot *100, 1)}%)" 
    tot = df["gender"].count()
    return f"{cnt}" if measure=="n" else f"{cnt} ({round(cnt/tot *100, 1)}%)"

df_ogtt = pd.DataFrame(columns=title_row)
df_ogtt.loc[0] = ["OGTT (mmol/L) categories", "n", cell("n", female), cell("n", male), cell("n", (male | female), all=True)]
df_ogtt.loc[1] = ["", "<7.7", cell("<7.7", female), cell("<7.7", male), cell("<7.7", (male | female), all=True)]
df_ogtt.loc[2] = ["", "7.8-11.1", cell("7.8-11.1", female), cell("7.8-11.1", male), cell("7.8-11.1", (male | female), all=True)]
df_ogtt.loc[3] = ["", "11.1 and above", cell(">=11.1", female), cell(">=11.1", male), cell(">=11.1", (male | female), all=True)]
df_ogtt.loc[4] = ["", "not done", cell("missing", female), cell("missing", male), cell("missing", (male | female), all=True)]


In [ ]:
# fbg and ogtt

def cell(measure, gender, all=None):
    if measure == "dm1":
        cond  = (df["ogtt"]>=11.1) | (df["fbg"]>=7.0) & (df["ogtt"].notna())
    elif measure == "other":
        cond  = ~((df["ogtt"]>=11.1) | (df["fbg"]>=7.0)) & (df["ogtt"].notna())
    elif measure == "ogtt":
        cond = (df["fbg"].notna()) & (df["ogtt"].isna())
    else:
        cond = (df["fbg"].notna())
    cnt = df.loc[gender & cond]["gender"].count()
    if not all:
        tot = df.loc[gender]["gender"].count()
        return f"{cnt}" if measure=="n" else f"{cnt} ({round(cnt/tot *100, 1)}%)" 
    tot = df["gender"].count()
    return f"{cnt}" if measure=="n" else f"{cnt} ({round(cnt/tot *100, 1)}%)"

df_fbg_ogtt = pd.DataFrame(columns=title_row)
df_fbg_ogtt.loc[0] = ["OGTT & FBG (mmol/L) categories", "n", cell("n", female), cell("n", male), cell("n", (male | female), all=True)]
df_fbg_ogtt.loc[1] = ["", "OGTT>=11.1 or FBG>=7.0", cell("dm1", female), cell("dm1", male), cell("dm1", (male | female), all=True)]
df_fbg_ogtt.loc[2] = ["", "other", cell("other", female), cell("other", male), cell("other", (male | female), all=True)]
df_fbg_ogtt.loc[3] = ["", "OGTT not done", cell("ogtt", female), cell("ogtt", male), cell("ogtt", (male | female), all=True)]


In [ ]:
df_table2 = pd.concat([df_gender, df_age, df_waist, df_art, df_bp, df_fbg, df_ogtt, df_fbg_ogtt], ignore_index=True)
df_table2

In [ ]:
# blood pressure
# Blood pressure interested in IQR25, IQR50(median), IQR75
df[["sys_blood_pressure_avg", "dia_blood_pressure_avg"]].describe()

In [ ]:
df_table

In [ ]:
# blood pressure


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import math
import seaborn as sns


In [ ]:
sns.boxplot(x="age_in_years",y="gender", data=df)

In [ ]:
sns.boxplot(x="fbg",y="gender", data=df)


In [ ]:
sns.boxplot(x="ogtt",y="gender", data=df)


In [ ]:
df[["age_in_years", "fbg", "ogtt"]].hist()

In [ ]:
sns.pairplot(df[["calculated_bmi_value", "fbg"]])

In [ ]:
cond = (df["fbg"]>=7.0) & (df["fbg"]<=10.0)
sns.displot(df[cond], x="fbg", hue="gender")

In [ ]:
sns.displot(df, x="sys_", hue="gender")